In [1]:
import pandas as pd
import numpy as np
import oracledb
from sqlalchemy import create_engine
import os
import glob
pd.set_option('display.max_columns',100)
import time

# A. Query Data

In [2]:
# credentials
username = os.environ['DBA_USR']
password = os.environ['DBA_PW']
host = '10.220.50.121'
port = '1661'
service_name = 'ANALYTIC'

In [3]:
connection_string = f'oracle+oracledb://{username}:{password}@{host}:{port}/?service_name={service_name}'

In [4]:
# create engine
engine = create_engine(connection_string)

In [ ]:
# create list of all dates
allDates = ['20241031','20240930','20240829','20240731','20240628','20240531',
            '20240430','20240329','20240229','20240131','20231229','20231130']
# no dpd for <202404

In [6]:
# open sql file
with open("CC Pull.sql", 'r') as file:
    query_template = file.read()

In [7]:
# print(query_template.format(dt='20241031'))

In [8]:
allMLs = []
for date in allDates:
    try:
        # print in case of errors
        print(f'Running {date}')

        # open and read the file as a single buffer
        sqlQuery = query_template.format(dt=date)

        # run query and store data in df
        print('SQL run')
        with engine.connect() as connection:
            ML = pd.read_sql(sqlQuery, connection) 
        print(f'{date} queried successfully')

        # change column names into UPPER CASE
        ML.columns = [x.upper() for x in ML.columns]
    except Exception as e:
        print(e)
        ML = pd.DataFrame()
    finally:
        allMLs.append(ML)

        time.sleep(5)

Running 20241031
SQL run
20241031 queried successfully
Running 20240930
SQL run
20240930 queried successfully
Running 20240829
SQL run
20240829 queried successfully
Running 20240731
SQL run
20240731 queried successfully
Running 20240628
SQL run
20240628 queried successfully
Running 20240531
SQL run
20240531 queried successfully
Running 20240430
SQL run
20240430 queried successfully
Running 20240329
SQL run
20240329 queried successfully
Running 20240229
SQL run
20240229 queried successfully
Running 20240131
SQL run
20240131 queried successfully
Running 20231229
SQL run
20231229 queried successfully
Running 20231130
SQL run
20231130 queried successfully


In [9]:
allMLs[1]

,BASE_DT,AGREE_ID,GCIF_NO,CIF_NO,NOTE_NO,PRD_TP,PRD_NM,PRD_SEGMENT,ORG_LMT_AMT,CURR_LMT_AMT,BAL,ORIG_INT_RT,CURR_INT_RT,OTH_CURR_INT_RT,CLCT_RTNG_FCL,ALLOW_PCT,ALLOW_PCT_ADD,RESTRUCT_CD,PASTDUE_SINCE_DT,PASTDUE_DAYS,WRITE_OFF_AMT,WRITE_OFF_DT,WRITE_OFF_YN,CC_STATUS,GCIF_NO,SEGMENT,SEGMENT_FUNDING,CUST_NM,OPEN_DATE,LAST_ACTIVE_DATE,CUST_TP,GENDER_CD,AGE,EMPLOYMENT_TYPE,MICRO_SEGMENT,CA_BAL,CA_AVG,SA_BAL,SA_AVG,TD_NOA,TD_BAL,TD_AVG,SY_FUNDING_BAL,SY_FUNDING_AVG,FUNDING_NOA,FUNDING_BAL,FUNDING_AVG,NOA_DORMANT,AUM,CC_PLAFOND,CC_BAL,PL_PLAFOND,PLOAN_BAL,TR_PLAFOND,TR_BAL,PB_PLAFOND,PB_BAL,PRK_PLAFOND,PRK_BAL,PPB_PLAFOND,PPB_BAL,BG_PLAFOND,BG_BAL,LC_PLAFOND,LC_BAL,SY_LOAN_PLAFOND,SY_LOAN_BAL,KPR_PLAFOND,KPR_BAL,KPM_PLAFOND,KPM_BAL,COLLECT,LUM,BA_UL_BAL,BA_ALL_BAL,MF_BAL,BONDS_BAL,TRB,M2U_TRX_TOTAL,M2U_TRXAMT_FIN,CRTRX_MTD,CRTRX_AMT_MTD,DBTRX_MTD,DBTRX_AMT_MTD
0,20240930,CP0010014423730004722008,G900455189,1120000000004534,4423730004722008,001,Credit Card,031100,7800000.0,9919780.00,80220.0,0.21,0.21,0.21,00001,0.01,0.0,00001,99991231,0,0.0,None,N,PERFORMING,G900455189,None,None,YENNI,2005-04-19,NaT,INDV,F,47.0,OTHERS,3.Mid Lifers,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,7800000.0,160440.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,160440.0,0.0,0.0,0.0,0.0,1.604400e+05,0.0,0.0,0.0,0.0,0.0,0.00
1,20240930,CP0010014423730007054110,G200315984,1010005000389119,4423730007054110,001,Credit Card,031100,4000000.0,27288784.17,0.0,0.21,0.21,0.21,00001,0.01,0.0,00001,99991231,0,0.0,None,N,PERFORMING,G200315984,None,None,HARRY WINATA,1992-03-20,NaT,INDV,M,74.0,SELF EMPLOYED,5.Golden Age,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,23000000.0,1267295.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6267295.0,0.0,0.0,0.0,0.0,6.267295e+06,0.0,0.0,0.0,0.0,0.0,0.00
2,20240930,CP0010014423730007115010,G500491659,1010010250225591,4423730007115010,001,Credit Card,031100,0.0,12250000.00,0.0,0.21,0.21,0.21,00001,0.01,0.0,00001,99991231,0,0.0,None,N,PERFORMING,G500491659,CFS-RB-3EA,CFS-RB-3EA,SUHARDI WAHID,2001-06-07,2024-09-30,INDV,M,53.0,SELF EMPLOYED,4.Silver Age,0.0,0.0,2.825477e+08,2.931556e+08,0.0,0.0,0.0,0.0,0.0,2.0,2.825477e+08,2.931556e+08,0.0,2.825477e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.825477e+08,7.0,20000000.0,0.0,0.0,3.0,20000002.16
3,20240930,CP0010014423730009799001,G600037923,1010001100409238,4423730009799001,001,Credit Card,031100,2500000.0,29310.20,0.0,0.21,0.21,0.21,00001,0.01,0.0,00001,99991231,0,0.0,None,N,PERFORMING,G600037923,None,None,ERY RADJALI,1997-03-13,NaT,INDV,F,51.0,OTHERS,4.Silver Age,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,45500000.0,45353449.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45353449.0,0.0,0.0,0.0,0.0,4.535345e+07,0.0,0.0,0.0,0.0,0.0,0.00
4,20240930,CP0010014423730014020021,G200447639,1060620000467761,4423730014020021,001,Credit Card,031100,9000000.0,28666666.67,0.0,0.21,0.21,0.21,00001,0.01,0.0,00001,99991231,0,0.0,None,N,PERFORMING,G200447639,None,None,"BENNY SUTANTO, SH",1994-04-21,NaT,INDV,M,65.0,OTHERS,5.Golden Age,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,9000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177992,20240930,CP001631464987001718

# B. EDA

In [10]:
# concat all MLs into one
MLs = pd.concat(allMLs, ignore_index=True)

C:\Users\MR38804\AppData\Local\Temp\ipykernel_1804\416592903.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  MLs = pd.concat(allMLs, ignore_index=True)


In [11]:
MLs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995100 entries, 0 to 1995099
Data columns (total 84 columns):
 #   Column            Dtype         
---  ------            -----         
 0   BASE_DT           object        
 1   AGREE_ID          object        
 2   GCIF_NO           object        
 3   CIF_NO            object        
 4   NOTE_NO           object        
 5   PRD_TP            object        
 6   PRD_NM            object        
 7   PRD_SEGMENT       object        
 8   ORG_LMT_AMT       float64       
 9   CURR_LMT_AMT      float64       
 10  BAL               float64       
 11  ORIG_INT_RT       float64       
 12  CURR_INT_RT       float64       
 13  OTH_CURR_INT_RT   float64       
 14  CLCT_RTNG_FCL     object        
 15  ALLOW_PCT         float64       
 16  ALLOW_PCT_ADD     float64       
 17  RESTRUCT_CD       object        
 18  PASTDUE_SINCE_DT  object        
 19  PASTDUE_DAYS      object        
 20  WRITE_OFF_AMT     float64       
 21  WRITE_OF

In [12]:
MLs.columns

Index(['BASE_DT', 'AGREE_ID', 'GCIF_NO', 'CIF_NO', 'NOTE_NO', 'PRD_TP',
       'PRD_NM', 'PRD_SEGMENT', 'ORG_LMT_AMT', 'CURR_LMT_AMT', 'BAL',
       'ORIG_INT_RT', 'CURR_INT_RT', 'OTH_CURR_INT_RT', 'CLCT_RTNG_FCL',
       'ALLOW_PCT', 'ALLOW_PCT_ADD', 'RESTRUCT_CD', 'PASTDUE_SINCE_DT',
       'PASTDUE_DAYS', 'WRITE_OFF_AMT', 'WRITE_OFF_DT', 'WRITE_OFF_YN',
       'CC_STATUS', 'GCIF_NO', 'SEGMENT', 'SEGMENT_FUNDING', 'CUST_NM',
       'OPEN_DATE', 'LAST_ACTIVE_DATE', 'CUST_TP', 'GENDER_CD', 'AGE',
       'EMPLOYMENT_TYPE', 'MICRO_SEGMENT', 'CA_BAL', 'CA_AVG', 'SA_BAL',
       'SA_AVG', 'TD_NOA', 'TD_BAL', 'TD_AVG', 'SY_FUNDING_BAL',
       'SY_FUNDING_AVG', 'FUNDING_NOA', 'FUNDING_BAL', 'FUNDING_AVG',
       'NOA_DORMANT', 'AUM', 'CC_PLAFOND', 'CC_BAL', 'PL_PLAFOND', 'PLOAN_BAL',
       'TR_PLAFOND', 'TR_BAL', 'PB_PLAFOND', 'PB_BAL', 'PRK_PLAFOND',
       'PRK_BAL', 'PPB_PLAFOND', 'PPB_BAL', 'BG_PLAFOND', 'BG_BAL',
       'LC_PLAFOND', 'LC_BAL', 'SY_LOAN_PLAFOND', 'SY_LOAN_BAL', 'KPR_PL

In [13]:
MLs = MLs.loc[:, ~MLs.columns.duplicated()].copy()

In [23]:
MLs[['MICRO_SEGMENT']].value_counts()

MICRO_SEGMENT       
3.Mid Lifers            572475
4.Silver Age            307413
5.Golden Age            194494
1.Young Professional     84008
2.Young Families         67164
0.Younger                 2478
Name: count, dtype: int64

In [25]:
MLs[(MLs['CC_STATUS']=='DELINQUENT') & (MLs['MICRO_SEGMENT'].isin(['4.Silver Age','5.Golden Age']))][['BASE_DT']].value_counts()

BASE_DT 
20240531    577
20240430    576
20240731    576
20240628    566
20240829    518
20240930    475
20241031    447
Name: count, dtype: int64

## 1. SAGA total monthly bad debt

In [19]:
monthly = pd.pivot_table(MLs[MLs['CC_STATUS']=='DELINQUENT'], 
                         values=['GCIF_NO','CURR_LMT_AMT','BAL'],
                         index=['BASE_DT'],
                         columns=['MICRO_SEGMENT'],
                         aggfunc={'GCIF_NO':'count',
                                  'CURR_LMT_AMT':'sum',
                                  'BAL':'sum'})

In [20]:
print(monthly)

                      BAL                                                      \
MICRO_SEGMENT   0.Younger 1.Young Professional 2.Young Families  3.Mid Lifers   
BASE_DT                                                                         
20240430       88500255.0         1.076458e+09     1.506634e+09  1.939014e+10   
20240531       88303750.0         1.115632e+09     1.644644e+09  1.938466e+10   
20240628       74760372.0         1.258743e+09     1.525908e+09  2.025802e+10   
20240731       74192874.0         1.413881e+09     1.763239e+09  1.950479e+10   
20240829       74056619.0         1.505060e+09     1.508116e+09  1.926079e+10   
20240930       73597484.0         1.750002e+09     1.743633e+09  1.808617e+10   
20241031              NaN         2.157753e+09     1.545042e+09  2.018930e+10   

                                          CURR_LMT_AMT                       \
MICRO_SEGMENT  4.Silver Age  5.Golden Age    0.Younger 1.Young Professional   
BASE_DT                        

1. Hitung total bad debt SAGA all month
2. Bandingkan bad debt SAGA vs all other CC
2. Bandingkan 